In [1]:
import os
import glob
import pydicom
import cv2
import numpy as np
from read_roi import read_roi_zip
import zipfile

In [2]:
def get_LUT_value(data, window, level):
    try:
        window = 350
    except TypeError:
        pass
    try:
        level = 50
    except TypeError:
        pass

    return np.piecewise(data,
                        [data <= (level - 0.5 - (window - 1) / 2),
                         data > (level - 0.5 + (window - 1) / 2)],
                        [0, 255, lambda data: ((data - (level - 0.5)) /
                         (window - 1) + 0.5) * (255 - 0)])

def CreateFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error:Creating dirextory."+directory)

In [11]:
dcm_path = 'D:/appendicitis/1_data_raw/data/No_lesion/' # dcm 파일 폴더
file_extention = '.png' # 파일 확장자
file_dcm_path = [] # jpg 저장할 환자 폴더
roi_zip_path=[]
dcm_file_name = os.listdir(dcm_path) # 환자 폴더
dcm_file_name.sort()

for i in range(len(dcm_file_name)):
    name = dcm_path + dcm_file_name[i] + '/'
    file_dcm_path.append(name)
    roi_zip = dcm_path + dcm_file_name[i] + '/' + dcm_file_name[i] + '.zip'
    roi_zip_path.append(roi_zip)
# print(len(file_dcm_path), file_dcm_path)
# print(len(roi_zip_path),roi_zip_path)

In [12]:
zip_list_all = [] # roi 이름 저장

for i in range(len(roi_zip_path)):
    zip_file = zipfile.ZipFile(roi_zip_path[i])
    zip_list = zip_file.namelist()
    zip_list.sort()
    zip_list_all.append(zip_list)

In [ ]:
dir_path='D:/appendicitis/1_data_raw/data/No_lesion_png_2/'
CreateFolder(dir_path)
for i in range(len(file_dcm_path)):
    dcm = glob.glob(file_dcm_path[i]+ '/' + '*.dcm')
    
    for k in range(len(zip_list_all[i])):
        roi_name = zip_list_all[i][k][:4]
        #print(roi_name)
        dcm_name = str(int(roi_name)-1).zfill(4)
        #print(dcm_name)
        path = file_dcm_path[i] + dcm_file_name[i] +'_'+ dcm_name + '.dcm'
        print(path)
        try:
            ds = pydicom.read_file(path)
        except:
            path = file_dcm_path[i] + dcm_file_name[i] + dcm_name + '.dcm'
            ds=pydicom.read_file(path)
        ArrayDicom = ds.pixel_array
        ArrayDicom = ds.RescaleSlope*ArrayDicom+ds.RescaleIntercept
        ArrayDicom = get_LUT_value(ArrayDicom, ds.WindowWidth, ds.WindowCenter) #설정은 함수에서 조절하기
        file_name = dcm_file_name[i] +'_'+ dcm_name
        CreateFolder(dir_path+dcm_file_name[i]+'/')
        cv2.imwrite(dir_path+dcm_file_name[i]+'/'+file_name+'.png', ArrayDicom)
#         cv2.imwrite(file_name+'.png', ArrayDicom)

# 영역 crop 

In [65]:
i=25
png_path='D:/appendicitis/No_lesion_png/'+dcm_file_name[i]+'/'

file = glob.glob(png_path+'*.png')

In [66]:
roi = read_roi_zip(roi_zip_path[i])
name = zip_list_all[i][k][:14]
roi[name]

{'type': 'rectangle',
 'left': 107,
 'top': 256,
 'width': 46,
 'height': 46,
 'arc_size': 0,
 'name': '0063-0279-0130',
 'position': 63}

In [67]:
img = cv2.imread(file[k])
left_top = roi[name]['left'], roi[name]['top']
right_bottom = left_top[0]+roi[name]['width'], left_top[1]+roi[name]['height']
print(left_top,right_bottom)

(107, 256) (153, 302)


In [112]:
crop=img.copy()
crop_img=crop[left_top[1]:right_bottom[1],left_top[0]:right_bottom[0]]

In [113]:
# crop_img=cv2.resize(crop_img, None,fx=3, fy=3)
# cv2.imshow('',crop_img)
# cv2.waitKey()
# cv2.destroyAllWindows()

In [114]:
CreateFolder('D:/appendicitis/data/')

# DICOM 파일 경로와 ROI zip 경로 저장 

In [30]:
dcm_path = 'D:/appendicitis/1_data_raw/data/No_lesion_png_2/'# png 파일 폴더
file_extention = '.png' # 파일 확장자
file_dcm_path = [] # jpg 저장할 환자 폴더
roi_zip_path=[]
dcm_file_name = os.listdir(dcm_path) # 환자 폴더
dcm_file_name.sort()

for i in range(len(dcm_file_name)):
    name = dcm_path + dcm_file_name[i] + '/'
    file_dcm_path.append(name)
    roi_zip = dcm_path + dcm_file_name[i] + '/' + dcm_file_name[i] + '.zip'
    roi_zip_path.append(roi_zip)
# print(len(file_dcm_path), file_dcm_path)
# print(len(roi_zip_path),roi_zip_path)

# ROI 정보 저장

In [31]:
zip_list_all = [] # roi 이름 저장
roi_zip_path=glob.glob('D:/appendicitis/1_data_raw/data/No_lesion_roi/'+'*.zip')
for i in range(len(roi_zip_path)):
    zip_file = zipfile.ZipFile(roi_zip_path[i])
    zip_list = zip_file.namelist()
    zip_list.sort()
    zip_list_all.append(zip_list)

# Crop

In [ ]:
import random
mask_path = 'D:/appendicitis/1_data_crop/No_lesion_crop_all' #저장할 경로
dir_path='D:/appendicitis/1_data_raw/data/No_lesion_png_2/' # 이미지 경로
CreateFolder(mask_path)

for i in range(len(zip_list_all)):
#     if len(zip_list_all[i])>=4:
#         for m in range(4):
#             k=random.randrange(0,len(zip_list_all[i])-1)
#             png_path=dir_path+dcm_file_name[i]+'/'
#             file = glob.glob(png_path+'*.png')
#            # roi = read_roi_zip(roi_zip_path[i])
#             name = zip_list_all[i][k][:14]
#             img = cv2.imread(file[k])
# #             left_top = roi[name]['left'], roi[name]['top']
# #             right_bottom = left_top[0]+roi[name]['width'], left_top[1]+roi[name]['height']
# #     #         print(left_top,right_bottom)
# #             crop=img.copy()
# #             crop_img=crop[left_top[1]:right_bottom[1],left_top[0]:right_bottom[0]]
#             dcm_name = str(int(zip_list_all[i][k][:4])-1).zfill(4)
#             file_name = dcm_file_name[i] +'_'+ dcm_name
#             print(file_name)
#             CreateFolder(mask_path+'/'+dcm_file_name[i]+'/')
#             cv2.imwrite(mask_path+'/'+dcm_file_name[i]+'/'+file_name+'.png', img)
#             del zip_list_all[i][k]
#             if len(zip_list_all[i])==1:
#                 break
#     else:
    for k in range(len(zip_list_all[i])):
        png_path=dir_path+dcm_file_name[i]+'/'
        file = glob.glob(png_path+'*.png')
        roi = read_roi_zip(roi_zip_path[i])
        name = zip_list_all[i][k][:14]
        img = cv2.imread(file[k])
        left_top = roi[name]['left'], roi[name]['top']
        right_bottom = left_top[0]+roi[name]['width'], left_top[1]+roi[name]['height']
#         print(left_top,right_bottom)
        crop=img.copy()
        crop_img=crop[left_top[1]:right_bottom[1],left_top[0]:right_bottom[0]]
        dcm_name = str(int(zip_list_all[i][k][:4])-1).zfill(4)
        file_name = dcm_file_name[i] +'_'+ dcm_name
        print(file_name)
        CreateFolder(mask_path+'/'+dcm_file_name[i]+'/')
        cv2.imwrite(mask_path+'/'+dcm_file_name[i]+'/'+file_name+'.png', crop_img)

# Random downsampling

In [ ]:
import random
mask_path = 'D:/appendicitis/3_detection/train_appendicitis_png/'
dir_path='D:/appendicitis/1_data_raw/data2/Appendicitis_png_2/'
CreateFolder(mask_path)

for i in range(len(zip_list_all)):
    if len(zip_list_all[i])>=4:
        for m in range(4):
            k=random.randrange(0,len(zip_list_all[i])-1)
            png_path=dir_path+dcm_file_name[i]+'/'
            file = glob.glob(png_path+'*.png')
           # roi = read_roi_zip(roi_zip_path[i])
            name = zip_list_all[i][k][:14]
            img = cv2.imread(file[k])
#             left_top = roi[name]['left'], roi[name]['top']
#             right_bottom = left_top[0]+roi[name]['width'], left_top[1]+roi[name]['height']
#     #         print(left_top,right_bottom)
#             crop=img.copy()
#             crop_img=crop[left_top[1]:right_bottom[1],left_top[0]:right_bottom[0]]
            dcm_name = str(int(zip_list_all[i][k][:4])-1).zfill(4)
            file_name = dcm_file_name[i] +'_'+ dcm_name
            print(file_name)
            CreateFolder(mask_path+'/'+dcm_file_name[i]+'/')
            cv2.imwrite(mask_path+'/'+dcm_file_name[i]+'/'+file_name+'.png', img)
            del zip_list_all[i][k]
            if len(zip_list_all[i])==1:
                break
    else:
        for k in range(len(zip_list_all[i])):
            png_path=dir_path+dcm_file_name[i]+'/'
            file = glob.glob(png_path+'*.png')
            #roi = read_roi_zip(roi_zip_path[i])
            name = zip_list_all[i][k][:14]
            img = cv2.imread(file[k])
#             left_top = roi[name]['left'], roi[name]['top']
#             right_bottom = left_top[0]+roi[name]['width'], left_top[1]+roi[name]['height']
#     #         print(left_top,right_bottom)
#             crop=img.copy()
#             crop_img=crop[left_top[1]:right_bottom[1],left_top[0]:right_bottom[0]]
#             dcm_name = str(int(zip_list_all[i][k][:4])-1).zfill(4)
            file_name = dcm_file_name[i] +'_'+ dcm_name
            print(file_name)
            CreateFolder(mask_path+'/'+dcm_file_name[i]+'/')
            cv2.imwrite(mask_path+'/'+dcm_file_name[i]+'/'+file_name+'.png', img)